### 解析SAS字典文件

In [100]:
import os
import re
sas_desc_file = r'/home/ghost/workspace/uda-dend-capstone/data-input/I94_SAS_Labels_Descriptions.SAS'
csv_out_path = r'/home/ghost/workspace/uda-dend-capstone/data-upload/csv'

def match_to_csv(match_var,csv_file_name):
    lines = []
    reg_exp = r'\s*=\s*'
    for line in match_var.split('\n'):
        if line.strip() and re.search(reg_exp,line):    # if not empty line
            line_t = re.sub(reg_exp,'|',line.strip().replace("'",''))    # replace xxx = 'yyy' to xxx|yyy
            lines.append(line_t + '\n')
    with open(csv_file_name,'w') as fw:
        fw.writelines(lines)
    
if not os.path.exists(csv_out_path):
        os.mkdir(csv_out_path)
        
# sas desc file is small,so read all into one variable 
with open(sas_desc_file,'r') as file: 
    sas_desc = file.read()

# reg_exp = r'\*/\s*value\s+(\S+)(.*?);'
# matchs = re.findall(reg_exp,sas_desc,re.S)

reg_exp = r'/\* I94VISA(.*?)\*/'
matches = re.findall(reg_exp,sas_desc,re.S)
match_to_csv(matches[0],os.path.join(csv_out_path,'i94visa.csv'))


# for m in matchs:
#     file_name = os.path.join(csv_out_path,m[0].replace('$','') + '.csv')   # removing leading $
#     match_to_csv(m[1],file_name)

In [11]:
matches[0]

' - Visa codes collapsed into three categories:\n   1 = Business\n   2 = Pleasure\n   3 = Student\n'

### 读取SAS文件，转换为parquet

In [1]:
import glob
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
    .enableHiveSupport().getOrCreate()

columns = [
    'cicid', 'i94yr', 'i94mon', 
    'i94cit', 'i94res', 'i94port',
    'arrdate', 'i94mode', 'i94addr',
    'depdate', 'i94bir', 'i94visa',
    'count', 'dtadfile', 'visapost',
    'occup', 'entdepa', 'entdepd',
    'entdepu', 'matflag', 'biryear',
    'dtaddto', 'gender', 'insnum',
    'airline', 'admnum', 'fltno',
    'visatype'
]

schema = """
    cicid	double,
    i94yr	double,
    i94mon	double,
    i94cit	double,
    i94res	double,
    i94port	varchar(200),
    arrdate	double,
    i94mode	double,
    i94addr	varchar(200),
    depdate	double,
    i94bir	double,
    i94visa	double,
    count	double,
    dtadfile	varchar(200),
    visapost	varchar(200),
    occup	varchar(200),
    entdepa	varchar(200),
    entdepd	varchar(200),
    entdepu	varchar(200),
    matflag	varchar(200),
    biryear	double,
    dtaddto	varchar(200),
    gender	varchar(200),
    insnum	varchar(200),
    airline	varchar(200),
    admnum	double,
    fltno	varchar(200),
    visatype	varchar(200)
"""
# read all sas7bdat file under data-input folder
df_spark = spark.createDataFrame([],schema) # create a empty dataframe
for filename in glob.glob('/home/ghost/workspace/uda-dend-capstone/data-input/*.sas7bdat'):
    df_sas = spark.read\
            .format('com.github.saurfang.sas.spark')\
            .schema(schema)\
            .load(filename)
    df_spark = df_spark.union(df_sas)

In [2]:
df_spark.show(5)

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|  6.0|2016.0|   4.0| 692.0| 692.0|    XXX|20573.0|   null|   null|   null|  37.0|    2.0|  1.0|    null|    null| null|      T|   null|      U|   null| 1979.0|10282016|  null|  null|   null| 1.897628485E9| null|      B2|
|  7.0|2016.0|   4.0| 254.0| 276.0|    ATL|20551.0|    1.0|     AL|   null|  25.0|    3.0|  1.0|20130811|     SE

In [3]:
%%time # about 8 mins
# write to parquet
df_spark.write.mode('overwrite').parquet("/home/ghost/workspace/uda-de-capstone/data-upload/sas_data")

CPU times: user 59.7 ms, sys: 12.1 ms, total: 71.8 ms
Wall time: 8min 6s


### 上传到S3

#### 方法一

In [ ]:
%%time 
# 200M parquet files,about 10 mins
import subprocess as sub

# params
upload_file = 'sas_data.parquet'

s3_copy_cmd = [
    'aws','s3','cp',
    f'/home/ghost/workspace/uda-dend-capstone/data-upload/{upload_file}',
    f's3://bucket-vincent-archive/{upload_file}',
    '--recursive','--exclude','*.crc'
]
result = sub.run(s3_copy_cmd,stdout=sub.PIPE,stderr=sub.PIPE)
if result.returncode:
    print(result.stderr)
    print('upload to s3 faild')
else:
    print('successfully upload files to s3.')

#### 方法二（建议采用此方法）

In [102]:
access_id = 'xxx'
secret_key = 'xxx'

In [109]:
%%time
import boto3
import os
import fnmatch

def upload_files(bucket,path,exclude):
    """upload files in path with exclude file pattern"""
    session = boto3.Session(
        aws_access_key_id = access_id,
        aws_secret_access_key = secret_key,
        region_name = 'us-west-2'
    )
    s3 = session.resource('s3')
    bucket = s3.Bucket(bucket)
    
    for subdir, dirs, files in os.walk(path):
        for file in files:
            full_path = os.path.join(subdir, file)
            if not fnmatch.fnmatch(full_path,exclude):
                with open(full_path, 'rb') as data:
                    relative_path = full_path[len(path.rstrip('/')) + 1:]
                    print(f'uploading {relative_path}')
                    bucket.put_object(Key = relative_path, Body = data)
                    print(f'{relative_path} uploaded.')
    print('all files uploaded.')

# test
# upload_files('bucket-vincent-archive','/home/ghost/workspace/uda-dend-capstone/data-upload','*.crc')
upload_files('bucket-vincent-archive','/home/ghost/workspace/uda-dend-capstone/data-upload/csv','*.crc')

uploading i94model.csv
i94model.csv uploaded.
uploading us_cities_demographics.csv
us_cities_demographics.csv uploaded.
uploading i94prtl.csv
i94prtl.csv uploaded.
uploading i94addrl.csv
i94addrl.csv uploaded.
uploading airport_codes.csv
airport_codes.csv uploaded.
uploading i94visa.csv
i94visa.csv uploaded.
uploading i94cntyl.csv
i94cntyl.csv uploaded.
all files uploaded.
CPU times: user 512 ms, sys: 140 ms, total: 651 ms
Wall time: 21.7 s


### 读 Parquet

In [114]:
import pyspark
from pyspark.sql import SparkSession

In [115]:
# 创建或获取会话
spark = SparkSession.builder.appName('Python Spark SQL example').getOrCreate()

In [120]:
df = spark.read.format('csv')\
    .option('header', 'true')\
    .option('inferSchema', 'true')\
    .option('sep', ';')\
    .load('/home/ghost/workspace/uda-dend-capstone/data-upload/csv/us_cities_demographics.csv')

# rename column,or space with rasise a error
for col_name in df.columns:
    df = df.withColumnRenamed(col_name,col_name.replace(' ','-'))

# write to parquet    
df.write.mode('overwrite').partitionBy('Race').parquet('/home/ghost/workspace/uda-dend-capstone/data-upload/test.parquet')

In [132]:
# df.write.mode('overwrite').partitionBy('Race').parquet('/home/ghost/workspace/uda-dend-capstone/data-upload/test.parquet')
df.columns

['City',
 'State',
 'Median-Age',
 'Male-Population',
 'Female-Population',
 'Total-Population',
 'Number-of-Veterans',
 'Foreign-born',
 'Average-Household-Size',
 'State-Code',
 'Race',
 'Count']

In [133]:
# 读 parquet
df_parquent = spark.read.format('parquet')\
    .load('/home/ghost/workspace/uda-dend-capstone/data-upload/test.parquet/Race=Asian/*.parquet')

In [134]:
# 可以发现仅仅只读单个的parquet，结果中是没有被分区的列的
df_parquent.columns

['City',
 'State',
 'Median-Age',
 'Male-Population',
 'Female-Population',
 'Total-Population',
 'Number-of-Veterans',
 'Foreign-born',
 'Average-Household-Size',
 'State-Code',
 'Count']

### 临时测试

In [6]:
x = 324
if x:
    print(1)

1


In [48]:
import os
import fnmatch
import shutil
import distutils.dir_util as dir_util
from datetime import datetime, timedelta
exclude = '*.crc'
full_path = 'sas_data/.part-00006-b951a2bb-402f-476d-97e3-84740450bd69-c000.snappy.parquet.crc'
full_path_2 = '546.crc'
print(fnmatch.fnmatch(full_path,exclude))
print(fnmatch.fnmatch(full_path_2,exclude))

True
True


In [11]:
os.getenv('HOME')

'/home/ghost'

In [43]:
x = 'a = {},b = {{}},c = {}'
x.format(1)

IndexError: tuple index out of range

In [50]:
x = {'a':1,'b':3}
x.update({'b':4,'c':5})
x

{'a': 1, 'b': 4, 'c': 5}

In [17]:
datedatetime.today()

datetime.datetime(2019, 10, 19, 15, 45, 7, 357878)

In [70]:
# clear input folder
def clear_folder(data_input):
    for name in os.listdir(data_input):
        file = os.path.join(data_input,name)
        try:
            dir_util.remove_tree(file)
        except OSError:
            os.remove(file)

In [86]:
data_input_archive = '/home/ghost/workspace/uda-dend-capstone/data-input-archive'
data_input = '/home/ghost/workspace/uda-dend-capstone/data-input'
fold_name = datetime.today().strftime('%Y%m%d')

# dir_util.copy_tree(data_input,os.path.join(data_input_archive,fold_name)) # copy file to archive folder
# clear_folder(data_input) # clear input folder
# clear_folder(upload)

In [51]:
# shutil.copytree(data_input,os.path.join(data_input_archive,fold_name))


['/home/ghost/workspace/uda-dend-capstone/data-input-archive/20191019/data-input.zip',
 '/home/ghost/workspace/uda-dend-capstone/data-input-archive/20191019/i94_feb16_sub.sas7bdat',
 '/home/ghost/workspace/uda-dend-capstone/data-input-archive/20191019/us-cities-demographics.csv',
 '/home/ghost/workspace/uda-dend-capstone/data-input-archive/20191019/immigration_data_sample.csv',
 '/home/ghost/workspace/uda-dend-capstone/data-input-archive/20191019/.ipynb_checkpoints/immigration_data_sample-checkpoint.csv',
 '/home/ghost/workspace/uda-dend-capstone/data-input-archive/20191019/.ipynb_checkpoints/airport-codes_csv-checkpoint.csv',
 '/home/ghost/workspace/uda-dend-capstone/data-input-archive/20191019/.ipynb_checkpoints/I94_SAS_Labels_Descriptions-checkpoint.SAS',
 '/home/ghost/workspace/uda-dend-capstone/data-input-archive/20191019/I94_SAS_Labels_Descriptions.SAS',
 '/home/ghost/workspace/uda-dend-capstone/data-input-archive/20191019/airport-codes_csv.csv']

In [68]:
# dir_util.remove_tree(data_input)
# shutil.rmtree(data_input)


In [96]:
# import glob
# from shutil import copy
# for name in glob.glob('/home/ghost/workspace/uda-dend-capstone/data-input/**/**.csv',recursive=True):
#     copy(name,os.path.join(data_input,'test'))

In [95]:
# copy('/home/ghost/workspace/uda-dend-capstone/data-input/airport_codes.csv','/home/ghost/workspace/uda-dend-capstone/data-input/test')

'/home/ghost/workspace/uda-dend-capstone/data-input/test/airport_codes.csv'

In [112]:
import os
# os.path.basename('/home/ghost/workspace/uda-dend-capstone/data-input/airport_codes.csv')
os.path.join('/home/ghost/workspace/uda-dend-capstone/data-input/','airport_codes.csv')

'/home/ghost/workspace/uda-dend-capstone/data-input/airport_codes.csv'

In [9]:
import json  
import pandas as pd  
from pandas.io.json import json_normalize

In [114]:
with open('/home/ghost/workspace/uda-dend-capstone/data-input/iso_3166_2.json') as f:
    country_region_json = json.load(f)

In [121]:
df_iso_country_region = country_region_json
df_iso_country_region = json_normalize(iso_country_region['CN'])
df_iso_country_region['country_code'] = 'CN'
df_iso_country_region

,name,divisions.CN-11,divisions.CN-50,divisions.CN-31,divisions.CN-12,divisions.CN-34,divisions.CN-35,divisions.CN-62,divisions.CN-44,divisions.CN-52,...,divisions.CN-71,divisions.CN-53,divisions.CN-33,divisions.CN-45,divisions.CN-15,divisions.CN-64,divisions.CN-65,divisions.CN-54,divisions.CN-91,country_code
0,China,Beijing,Chongqing,Shanghai,Tianjin,Anhui,Fujian,Gansu,Guangdong,Guizhou,...,Taiwan,Yunnan,Zhejiang,Guangxi,Nei Monggol,Ningxia,Xinjiang,Xizang,Hong Kong,CN


In [94]:
unpivot_cols = [col for col in list(df_iso_country_region) if col not in ('name','country_code')]
df_iso_country_region = pd.melt(df_iso_country_region,id_vars=['country_code','name'],value_vars=unpivot_cols).head()

In [97]:
df_iso_country_region['variable'] = df_iso_country_region['variable'].str.replace('divisions.','')

In [98]:
df_iso_country_region

,country_code,name,variable,value
0,CN,China,CN-11,Beijing
1,CN,China,CN-50,Chongqing
2,CN,China,CN-31,Shanghai
3,CN,China,CN-12,Tianjin
4,CN,China,CN-34,Anhui


所以方法就是一个个解析，然后合并

In [ ]:
country_region_json

### 解析奇怪的嵌套JSON

In [123]:
import json  
import pandas as pd  
from pandas.io.json import json_normalize

with open('/home/ghost/workspace/uda-dend-capstone/data-input/iso_3166_2.json') as f:
    country_region_json = json.load(f)

cols = ['country_code','country_name','region_code','region_name']
df_iso_country_region = pd.DataFrame(columns = cols)

for country_code in country_region_json:
    df_country_region = json_normalize(iso_country_region[country_code])
    df_country_region['country_code'] = country_code
    df_country_region.rename(columns = {'name':'country_name'},inplace = True)
    unpivot_cols = [col for col in list(df_country_region) if col not in ('country_name','country_code')]
    df_country_region = pd.melt(df_country_region,id_vars=['country_code','country_name'],value_vars = unpivot_cols
                               ,var_name = 'region_code',value_name = 'region_name')
    df_country_region['region_code'] = df_country_region['region_code'].str.replace('divisions.','')
    df_iso_country_region = df_iso_country_region.append(df_country_region,ignore_index = True)

In [135]:
df_iso_country_region.to_json('/home/ghost/workspace/uda-dend-capstone/data-upload/iso_country_region.json',orient = 'table',index = False)

In [137]:
df_iso_country_region.to_csv('/home/ghost/workspace/uda-dend-capstone/data-upload/iso_country_region.csv',sep='|',index=False)